# Train Model with DPO - Phi 2

In [1]:
!pip install -q  datasets trl accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from huggingface_hub import login
hf_token = input("Please enter your Hugging Face API Key:")
login(token=hf_token)

In [4]:
dataset = load_dataset("Eshita-ds/cot-llm-dpo-dataset-final")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/649 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/38.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/270 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/30 [00:00<?, ? examples/s]

In [5]:
model_name = "microsoft/phi-2"

# model = AutoModelForCausalLM.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.gradient_checkpointing_enable()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # set pad token
tokenizer.chat_template = "{% for message in messages %}{{ message['content'] }}\n{% endfor %}"

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [6]:
def format_chat_prompt(user_input, system_message="You are a helpful AI Tutor."):

    # Format user message
    user_prompt = f"<|chat_start|>user\n{user_input}<|chat_end|>\n"

    # Start assistant's turn
    tutor_prompt = "<|chat_start|>tutor\n"

    # Combine prompts
    formatted_prompt = user_prompt + tutor_prompt

    return formatted_prompt

In [7]:
# Set up text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example prompt
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=700, truncation=True, num_return_sequences=1, temperature=0.7)

print(outputs[0]['generated_text'])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<|chat_start|>user
You are an AI tutor that thinks and provides detailed and step-by-step explanations for the provided maths question.

**Question**: a circle of radius 1 is internally tangent to two circles of radius 2 at points  $a$ and $b$, where $ab$ is a diameter of the smaller circle. what is the  area of the region, shaded in the figure, that is outside  the smaller circle and inside each of the two larger circles? express your answer in terms of $\\pi$ and in simplest radical form.\n\n[asy]\nunitsize(1cm);\npair a = (0,-1), b = (0,1);\nfill(arc(a,2,30,90)--arc((0,0),1,90,-90)--arc(b,2,270,330)--cycle,gray(0.7));\nfill(arc(a,2,90,150)--arc(b,2,210,270)--arc((0,0),1,270,90)--cycle,gray(0.7));\ndraw(circle((0,-1),2));\ndraw(circle((0,1),2));\ndraw(circle((0,0),1));\ndraw((0,0)--(0.71,0.71),arrow);\ndraw((0,-1)--(-1.41,-2.41),arrow);\ndraw((0,1)--(1.41,2.41),arrow);\ndot((0,-1));\ndot((0,1));\nlabel("$a$",a,s);\nlabel("$b$",b,n);\nlabel("2",(0.7,1.7),n);\nlabel("2",(-0.7,-1.7),n);

In [8]:
ft_model_name = model_name.split('/')[1].replace("Instruct", "DPO")

training_args = DPOConfig(
    output_dir=ft_model_name,
    logging_steps=25,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
)

device = torch.device('cuda')

In [9]:
trainer = DPOTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
)
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shtgupta8 (shtgupta8-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
0,0.692400,0.686979,0.001725,-0.026815,0.300000,0.028271,-203.466660,-176.066666,1.279948,1.290104
1,0.704500,0.697917,0.036646,0.038444,0.366667,-0.001717,-203.266663,-175.133331,1.276042,1.282031
2,0.700000,0.701823,0.038379,0.046761,0.433333,-0.008390,-202.899994,-175.199997,1.276823,1.296354


TrainOutput(global_step=99, training_loss=0.6952335858585859, metrics={'train_runtime': 408.101, 'train_samples_per_second': 1.985, 'train_steps_per_second': 0.243, 'total_flos': 0.0, 'train_loss': 0.6952335858585859, 'epoch': 2.977777777777778})

In [10]:
# Load the fine-tuned model
ft_model = trainer.model

In [11]:
model_id = f"Eshita-ds/{ft_model_name}-DPO"
trainer.push_to_hub(model_id)

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

events.out.tfevents.1743825212.2bfcadb8c58a.1482.0:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/6.26k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Eshita-ds/phi-2/commit/7584532ecfc30796344314ae8df275a57a16ecec', commit_message='Eshita-ds/phi-2-DPO', commit_description='', oid='7584532ecfc30796344314ae8df275a57a16ecec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Eshita-ds/phi-2', endpoint='https://huggingface.co', repo_type='model', repo_id='Eshita-ds/phi-2'), pr_revision=None, pr_num=None)

In [12]:
# Set up text generation pipeline
generator = pipeline("text-generation", model=ft_model, tokenizer=tokenizer)

# Example prompt
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=700, truncation=True, num_return_sequences=1, temperature=0.7)

print(outputs[0]['generated_text'])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<|chat_start|>user
You are an AI tutor that thinks and provides detailed and step-by-step explanations for the provided maths question.

**Question**: a circle of radius 1 is internally tangent to two circles of radius 2 at points  $a$ and $b$, where $ab$ is a diameter of the smaller circle. what is the  area of the region, shaded in the figure, that is outside  the smaller circle and inside each of the two larger circles? express your answer in terms of $\\pi$ and in simplest radical form.\n\n[asy]\nunitsize(1cm);\npair a = (0,-1), b = (0,1);\nfill(arc(a,2,30,90)--arc((0,0),1,90,-90)--arc(b,2,270,330)--cycle,gray(0.7));\nfill(arc(a,2,90,150)--arc(b,2,210,270)--arc((0,0),1,270,90)--cycle,gray(0.7));\ndraw(circle((0,-1),2));\ndraw(circle((0,1),2));\ndraw(circle((0,0),1));\ndraw((0,0)--(0.71,0.71),arrow);\ndraw((0,-1)--(-1.41,-2.41),arrow);\ndraw((0,1)--(1.41,2.41),arrow);\ndot((0,-1));\ndot((0,1));\nlabel("$a$",a,s);\nlabel("$b$",b,n);\nlabel("2",(0.7,1.7),n);\nlabel("2",(-0.7,-1.7),n);